In [2]:
#!pip install --upgrade mysql-connector-python

In [1]:
#Importing Libraries
import pandas as pd
import mysql.connector
from mysql.connector import Error

## MySQL connection

In [3]:
import mysql.connector
from mysql.connector import Error

def execute_query(query):
    try:
        connection = mysql.connector.connect(
            host='127.0.0.1',
            database='Data_vista',
            user='root',
            password='root'
        )
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute(query)
            connection.commit()
            print("Query executed successfully")
            cursor.close()  # Close cursor after execution
    except Error as e:
        print(f"Error: {e}")
    finally:
        if 'connection' in locals() and connection.is_connected():
            connection.close()
            print("Connection closed")

# Example usage:
query = "CREATE TABLE IF NOT EXISTS TestTable (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255))"
execute_query(query)


Query executed successfully
Connection closed


In [4]:
file_path = "C:\\Users\\lenovo\\Downloads\\data_vista\\Final_Dataset.xlsx"
customer_data = pd.read_excel(file_path, sheet_name='Customer_Data')
sales_data = pd.read_excel(file_path, sheet_name='Sales_Data')

# Table Ceation

In [15]:
create_tables_queries = [
    """
    CREATE TABLE UserRole (
        RoleID VARCHAR(30) PRIMARY KEY,
        RoleName VARCHAR(50)
    )
    """,
    """
    CREATE TABLE Reason (
        ReasonID VARCHAR(30) PRIMARY KEY,
        ReasonDesc VARCHAR(255)
    )
    """,
    """
    CREATE TABLE Category (
        ProductCategoryID VARCHAR(30) PRIMARY KEY,
        CategoryName VARCHAR(50)
    )
    """,
    """
    CREATE TABLE Product (
        ProductID VARCHAR(30) PRIMARY KEY,
        ProductCategoryID VARCHAR(30),
        ProductName VARCHAR(255),
        PricePerUnit NUMERIC(10, 2),
        FOREIGN KEY (ProductCategoryID) REFERENCES Category(ProductCategoryID)
    )
    """,
    """
    CREATE TABLE Customer (
        CustomerID VARCHAR(30) PRIMARY KEY,
        FirstName VARCHAR(50),
        LastName VARCHAR(50),
        Age INT,
        Sex VARCHAR(10),
        Address VARCHAR(255)
    )
    """,
    """
    CREATE TABLE Location (
        CustomerID VARCHAR(30) ,
        City VARCHAR(50),
        State VARCHAR(50),
        ZipCode VARCHAR(20),
        FOREIGN KEY (CustomerID) REFERENCES Customer(CustomerID)
    )
    """,
    """
    CREATE TABLE `order_details` (
        OrderID VARCHAR(30) PRIMARY KEY,
        CustomerID VARCHAR(30),
        ProductID VARCHAR(30),
        OrderDate DATETIME,
        Quantity INT,
        TotalCost NUMERIC(10, 2),
        ShipDate DATETIME,
        DeliveredDate DATETIME,
        ModeOfPayment VARCHAR(20),
        OrderReturned BOOL,
        ReasonID VARCHAR(30),
        CustomerRating INT,
        FOREIGN KEY (CustomerID) REFERENCES Customer(CustomerID),
        FOREIGN KEY (ProductID) REFERENCES Product(ProductID),
        FOREIGN KEY (ReasonID) REFERENCES Reason(ReasonID)
    )
    """,
    """
    CREATE TABLE `User` (
        UserID VARCHAR(30) PRIMARY KEY,
        RoleID VARCHAR(30),
        Username VARCHAR(50),
        Password VARCHAR(50),
        FirstName VARCHAR(50),
        LastName VARCHAR(50),
        FOREIGN KEY (RoleID) REFERENCES UserRole(RoleID)
    )
    """
]
for query in create_tables_queries:
    execute_query(query)


Query executed successfully
Connection closed
Query executed successfully
Connection closed
Query executed successfully
Connection closed
Query executed successfully
Connection closed
Query executed successfully
Connection closed
Query executed successfully
Connection closed
Query executed successfully
Connection closed
Query executed successfully
Connection closed


# Loading Data into Database

In [16]:
import pandas as pd
import mysql.connector
from mysql.connector import Error

# Load data from Excel files
#sales_data.drop_duplicates(subset=['Return Reason'], inplace=True)

# Connect to MySQL database
try:
    connection = mysql.connector.connect(host='127.0.0.1',
                                         database='Data_vista',
                                         user='root',
                                         password='root')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()

        for i, row in sales_data.drop_duplicates(subset=['Return Reason'], inplace=False).iterrows():
            cursor.execute(f"INSERT INTO Reason (ReasonID, ReasonDesc) "
                           f"VALUES ('{row['Reason ID']}', '{row['Return Reason']}')")
        connection.commit()
        print("Data inserted into Reason table successfully.")

        # Load data into Category table
        for i, row in sales_data.drop_duplicates(subset=['Category ID'], inplace=False).iterrows():
            cursor.execute(f"INSERT INTO Category (ProductCategoryID, CategoryName) "
                           f"VALUES ('{row['Category ID']}', '{row['Product Category']}')")
        connection.commit()
        print("Data inserted into Category table successfully.")

        # Load data into Product table
        for i, row in sales_data.drop_duplicates(subset=['Product ID'], inplace=False).iterrows():
            cursor.execute(f"INSERT INTO Product (ProductID, ProductCategoryID, ProductName, PricePerUnit) "
                           f"VALUES ('{row['Product ID']}', '{row['Category ID']}', '{row['Product Name']}', "
                           f"{row['Price per Unit']})")
        connection.commit()
        print("Data inserted into Product table successfully.")

        # Load data into Customer table
        for i, row in customer_data.drop_duplicates(subset=['Customer ID'], inplace=False).iterrows():
            cursor.execute(f"INSERT INTO Customer (CustomerID, FirstName, LastName, Age, Sex, Address) "
                           f"VALUES ('{row['Customer ID']}', '{row['First Name']}', '{row['Last Name']}', "
                           f"{row['Age']}, '{row['Sex']}', '{row['Address']}')")
        connection.commit()
        print("Data inserted into Customer table successfully.")

        # Load data into Location table
        for i, row in customer_data.drop_duplicates(subset=['Customer ID'], inplace=False).iterrows():
            cursor.execute(f"INSERT INTO Location (CustomerID, City, State, ZipCode) "
                           f"VALUES ('{row['Customer ID']}', '{row['City']}', '{row['State']}', '{row['Zip Code']}')")
        connection.commit()
        print("Data inserted into Location table successfully.")

        # Load data into OrderDetails table
        for i, row in sales_data.iterrows():
            # Convert string values to boolean
            order_returned = 1 if row['Order Returned'] == 'Yes' else 0
            query = """
                INSERT INTO order_details (OrderID, CustomerID, ProductID, OrderDate, Quantity, TotalCost, ShipDate, DeliveredDate, 
                                     ModeOfPayment, OrderReturned, ReasonID, CustomerRating) 
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            values = (
                str(row['Order ID']), str(row['Customer ID']), str(row['Product ID']), row['Order Date'], int(row['Quantity']), 
                float(row['Total Cost']), row['Ship Date'], row['Delivered Date'], str(row['Mode of Payment']), order_returned, 
                str(row['Reason ID']), int(row['Customer Rating (out of 5)'])
            )
            cursor.execute(query, values)
        connection.commit()
        print("Data inserted into OrderDetails table successfully.")

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed.")


Connected to MySQL Server version  8.3.0
Data inserted into Reason table successfully.
Data inserted into Category table successfully.
Data inserted into Product table successfully.
Data inserted into Customer table successfully.
Data inserted into Location table successfully.
Data inserted into OrderDetails table successfully.
MySQL connection is closed.
